In [19]:
# Load libraries
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

In [20]:
df = pd.read_csv("Placement_Data_Full_Class.csv",  usecols=['gender', 'ssc_p', 'ssc_b', 'hsc_p', 'hsc_b', 'hsc_s', 'degree_p', 'degree_t', 'workex', 'etest_p', 'mba_p', 'status', 'specialisation'])

In [21]:
df

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status
0,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed
1,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed
2,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed
3,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed
4,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,M,80.60,Others,82.00,Others,Commerce,77.60,Comm&Mgmt,No,91.0,Mkt&Fin,74.49,Placed
211,M,58.00,Others,60.00,Others,Science,72.00,Sci&Tech,No,74.0,Mkt&Fin,53.62,Placed
212,M,67.00,Others,67.00,Others,Commerce,73.00,Comm&Mgmt,Yes,59.0,Mkt&Fin,69.72,Placed
213,F,74.00,Others,66.00,Others,Commerce,58.00,Comm&Mgmt,No,70.0,Mkt&HR,60.23,Placed


In [22]:
df.isnull().sum()

gender            0
ssc_p             0
ssc_b             0
hsc_p             0
hsc_b             0
hsc_s             0
degree_p          0
degree_t          0
workex            0
etest_p           0
specialisation    0
mba_p             0
status            0
dtype: int64

In [23]:
print(df["status"].value_counts())
print(df["status"].value_counts(normalize = True))

Placed        148
Not Placed     67
Name: status, dtype: int64
Placed        0.688372
Not Placed    0.311628
Name: status, dtype: float64


In [24]:
df = df.fillna(0)
df['status'] = df['status'].apply(lambda x:1 if x == 'Placed' else 0)

df['ssc_b'] = df['ssc_b'].apply(lambda x:1 if x == 'Central' else 0)
df['hsc_b'] = df['hsc_b'].apply(lambda x:1 if x == 'Central'  else 0)


df['science'] = df['hsc_s'].apply(lambda x:1 if x == 'Science'  else 0)
df['arts'] = df['hsc_s'].apply(lambda x:1 if x == 'Arts' else 0)
df['commerce'] = df['hsc_s'].apply(lambda x:1 if x == 'Commerce' else 0)



df['gender'] = df['gender'].apply(lambda x:1 if x == 'M' else 0)

df['workex'] = df['workex'].apply(lambda x:1 if x == 'Yes' else 0)

df['specialisation'] = df['specialisation'].apply(lambda x:1 if x == 'Mkt&HR' else 0)

df['degree_t'] = df['degree_t'].apply(lambda x:1 if x == 'Sci&Tech' else 0)

In [25]:
df

,gender,ssc_p,ssc_b,hsc_p,hsc_b,hsc_s,degree_p,degree_t,workex,etest_p,specialisation,mba_p,status,science,arts,commerce
0,1,67.00,0,91.00,0,Commerce,58.00,1,0,55.0,1,58.80,1,0,0,1
1,1,79.33,1,78.33,0,Science,77.48,1,1,86.5,0,66.28,1,1,0,0
2,1,65.00,1,68.00,1,Arts,64.00,0,0,75.0,0,57.80,1,0,1,0
3,1,56.00,1,52.00,1,Science,52.00,1,0,66.0,1,59.43,0,1,0,0
4,1,85.80,1,73.60,1,Commerce,73.30,0,0,96.8,0,55.50,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,0,82.00,0,Commerce,77.60,0,0,91.0,0,74.49,1,0,0,1
211,1,58.00,0,60.00,0,Science,72.00,1,0,74.0,0,53.62,1,1,0,0
212,1,67.00,0,67.00,0,Commerce,73.00,0,1,59.0,0,69.72,1,0,0,1
213,0,74.00,0,66.00,0,Commerce,58.00,0,0,70.0,1,60.23,1,0,0,1


In [26]:
df = df.drop(['hsc_s'], axis=1)

In [27]:
st = df['status']
rest = df.drop(['status'], axis=1)

In [28]:
rest

,gender,ssc_p,ssc_b,hsc_p,hsc_b,degree_p,degree_t,workex,etest_p,specialisation,mba_p,science,arts,commerce
0,1,67.00,0,91.00,0,58.00,1,0,55.0,1,58.80,0,0,1
1,1,79.33,1,78.33,0,77.48,1,1,86.5,0,66.28,1,0,0
2,1,65.00,1,68.00,1,64.00,0,0,75.0,0,57.80,0,1,0
3,1,56.00,1,52.00,1,52.00,1,0,66.0,1,59.43,1,0,0
4,1,85.80,1,73.60,1,73.30,0,0,96.8,0,55.50,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,0,82.00,0,77.60,0,0,91.0,0,74.49,0,0,1
211,1,58.00,0,60.00,0,72.00,1,0,74.0,0,53.62,1,0,0
212,1,67.00,0,67.00,0,73.00,0,1,59.0,0,69.72,0,0,1
213,0,74.00,0,66.00,0,58.00,0,0,70.0,1,60.23,0,0,1


In [29]:
st

0      1
1      1
2      1
3      0
4      1
      ..
210    1
211    1
212    1
213    1
214    0
Name: status, Length: 215, dtype: int64

In [30]:
df = pd.concat([rest, st], axis=1)

In [31]:
df

,gender,ssc_p,ssc_b,hsc_p,hsc_b,degree_p,degree_t,workex,etest_p,specialisation,mba_p,science,arts,commerce,status
0,1,67.00,0,91.00,0,58.00,1,0,55.0,1,58.80,0,0,1,1
1,1,79.33,1,78.33,0,77.48,1,1,86.5,0,66.28,1,0,0,1
2,1,65.00,1,68.00,1,64.00,0,0,75.0,0,57.80,0,1,0,1
3,1,56.00,1,52.00,1,52.00,1,0,66.0,1,59.43,1,0,0,0
4,1,85.80,1,73.60,1,73.30,0,0,96.8,0,55.50,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,1,80.60,0,82.00,0,77.60,0,0,91.0,0,74.49,0,0,1,1
211,1,58.00,0,60.00,0,72.00,1,0,74.0,0,53.62,1,0,0,1
212,1,67.00,0,67.00,0,73.00,0,1,59.0,0,69.72,0,0,1,1
213,0,74.00,0,66.00,0,58.00,0,0,70.0,1,60.23,0,0,1,1


In [32]:
# Split-out validation dataset
X = df.iloc[:,0:14].values
y = df.iloc[:,14].values
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)

In [33]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(kernel='linear', C=5)))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, x_train, y_train, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.848739 (0.028896)
LDA: 0.901008 (0.024060)
KNN: 0.859832 (0.066325)
CART: 0.824874 (0.065526)
NB: 0.808571 (0.066901)
SVM: 0.889412 (0.034561)


In [34]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(kernel='linear', C=5)))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, x_train, y_train, scoring='precision')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.881964 (0.038870)
LDA: 0.916741 (0.047253)
KNN: 0.862689 (0.061855)
CART: 0.861010 (0.065290)
NB: 0.863917 (0.063491)
SVM: 0.914843 (0.049541)


In [35]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(kernel='linear', C=5)))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, x_train, y_train, scoring='recall')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.909333 (0.047942)
LDA: 0.950333 (0.031277)
KNN: 0.958667 (0.026361)
CART: 0.867667 (0.031510)
NB: 0.867667 (0.041077)
SVM: 0.933667 (0.020827)


In [36]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(kernel='linear', C=5)))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, x_train, y_train, scoring='f1')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

LR: 0.893949 (0.021341)
LDA: 0.931591 (0.013098)
KNN: 0.907145 (0.040855)
CART: 0.868179 (0.034440)
NB: 0.864914 (0.046476)
SVM: 0.922960 (0.020932)
